In [1]:
from urllib.request import urlopen
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time
from contextlib import suppress

from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC


In [2]:
def subscrape_data(url, driver):
    # 12 - posted_by 
    # 13 - object_type 
    # 14 - condition 
    # 15 - heating 
    # 16 - floor 
    # 17 - max_floors

    # 19 - flag_available_immediately 
    # 20 - flag_legal 
    # 21 - flag_not_last_floor 
    # 22 - flag_not_in_house
    # 23 - flag_VAT_return
    # 25 - flag_duplex
    # 29 - flag_energy_saving
    # 31 - flag_balcony
    # 33 - flag_covered_balcony
    # 34 - flag_air_condition
    # 35 - flag_elevator
    # 36 - flag_has_basement
    # 38 - flag_telephone
    # 39 - flag_cable_tv
    # 40 - flag_internet
    # 41 - flag_interphone
    # 42 - flag_surveliance
    # 44 - flag_garage
    # 45 - flag_parking
    

    driver.get(url)
    time.sleep(1)      # wait page to load, to grab raw values
    page=driver.page_source
    soup = BeautifulSoup(page, 'html.parser')

    data = {}

    
    try:    
        data['Posted_By'] = soup.find('span',attrs={'id':'plh12'}).text
    except:
        data['Posted_By'] = ''
    try:    
        data['Object_Type'] = soup.find('span',attrs={'id':'plh13'}).text
    except:
        data['Object_Type'] = ''
    try:    
        data['Condition'] = soup.find('span',attrs={'id':'plh14'}).text
    except:
        data['Condition'] = ''
    try:    
        data['Heating'] = soup.find('span',attrs={'id':'plh15'}).text
    except:
        data['Heating'] = ''
    try:    
        data['Floor'] = str(soup.find('span',attrs={'id':'plh16'}).text)
    except:
        data['Floor'] = '-1'
    try:    
        data['Max_Floors'] = str(soup.find('span',attrs={'id':'plh17'}).text)
    except:
        data['Max_Floors'] = '-1'
    
        
    # flags part
    try:
        soup.find('span',attrs={'id':'plh19'}).text
        data['Flag_Available_Immediately'] = 1
    except:
        data['Flag_Available_Immediately'] = -1
    
    try:
        soup.find('span',attrs={'id':'plh20'}).text
        data['Flag_Legal'] = 1
    except:
        data['Flag_Legal'] = -1
    try:
        soup.find('span',attrs={'id':'plh21'}).text
        data['Flag_Not_Last_Floor'] = 1
    except:
        data['Flag_Not_Last_Floor'] = -1
    try:
        soup.find('span',attrs={'id':'plh22'}).text
        data['Flag_Not_Part_of_House'] = 1
    except:
        data['Flag_Not_Part_of_House'] = -1
    try:
        soup.find('span',attrs={'id':'plh23'}).text
        data['Flag_VAT_return'] = 1
    except:
        data['Flag_VAT_return'] = -1
    try:
        soup.find('span',attrs={'id':'plh24'}).text
        data['Flag_24'] = 1
    except:
        data['Flag_24'] = -1
    try:
        soup.find('span',attrs={'id':'plh25'}).text
        data['Flag_Duplex'] = 1
    except:
        data['Flag_Duplex'] = -1
    try:
        soup.find('span',attrs={'id':'plh26'}).text
        data['Flag_26'] = 1
    except:
        data['Flag_26'] = -1
    try:
        soup.find('span',attrs={'id':'plh27'}).text
        data['Flag_27'] = 1
    except:
        data['Flag_27'] = -1
    try:
        soup.find('span',attrs={'id':'plh28'}).text
        data['Flag_28'] = 1
    except:
        data['Flag_28'] = -1
    try:
        soup.find('span',attrs={'id':'plh29'}).text
        data['Flag_Energy_Saving'] = 1
    except:
        data['Flag_Energy_Saving'] = -1
    try:
        soup.find('span',attrs={'id':'plh30'}).text
        data['Flag_30'] = 1
    except:
        data['Flag_30'] = -1
    try:
        soup.find('span',attrs={'id':'plh31'}).text
        data['Flag_Balcony'] = 1
    except:
        data['Flag_Balcony'] = -1
    try:
        soup.find('span',attrs={'id':'plh32'}).text
        data['Flag_32'] = 1
    except:
        data['Flag_32'] = -1
    try:
        soup.find('span',attrs={'id':'plh33'}).text
        data['Flag_Covered_Balcony'] = 1
    except:
        data['Flag_Covered_Balcony'] = -1
    try:
        soup.find('span',attrs={'id':'plh34'}).text
        data['Flag_AirConditioning'] = 1
    except:
        data['Flag_AirConditioning'] = -1
    try:
        soup.find('span',attrs={'id':'plh35'}).text
        data['Flag_Elevator'] = 1
    except:
        data['Flag_Elevator'] = -1
    try:
        soup.find('span',attrs={'id':'plh36'}).text
        data['Flag_Has_Basement'] = 1
    except:
        data['Flag_Has_Basement'] = -1
    try:
        soup.find('span',attrs={'id':'plh37'}).text
        data['Flag_37'] = 1
    except:
        data['Flag_37'] = -1
    try:
        soup.find('span',attrs={'id':'plh38'}).text
        data['Flag_Telephone'] = 1
    except:
        data['Flag_Telephone'] = -1
    try:
        soup.find('span',attrs={'id':'plh39'}).text
        data['Flag_CableTV'] = 1
    except:
        data['Flag_CableTV'] = -1
    try:
        soup.find('span',attrs={'id':'plh40'}).text
        data['Flag_Internet'] = 1
    except:
        data['Flag_Internet'] = -1
    try:
        soup.find('span',attrs={'id':'plh41'}).text
        data['Flag_Interphone'] = 1
    except:
        data['Flag_Interphone'] = -1
    try:
        soup.find('span',attrs={'id':'plh42'}).text
        data['Flag_Surveliance'] = 1
    except:
        data['Flag_Surveliance'] = -1
    try:
        soup.find('span',attrs={'id':'plh43'}).text
        data['Flag_43'] = 1
    except:
        data['Flag_43'] = -1
    try:
        soup.find('span',attrs={'id':'plh44'}).text
        data['Flag_Garage'] = 1
    except:
        data['Flag_Garage'] = -1
    try:
        soup.find('span',attrs={'id':'plh45'}).text
        data['Flag_Parking'] = 1
    except:
        data['Flag_Parking'] = -1
    try:    
        data['Number_of_Busses'] = len(soup.findAll('div',attrs={'class':'city-lines'})[0].findAll('li',attrs={'class':'busses'}))
        data['Number_of_Trams'] = len(soup.findAll('div',attrs={'class':'city-lines'})[0].findAll('li',attrs={'class':'trams'}))
        data['Number_of_Trolleys'] = len(soup.findAll('div',attrs={'class':'city-lines'})[0].findAll('li',attrs={'class':'trolleybus'})) 
    except:
        data['Number_of_Busses'] = -1
        data['Number_of_Trams'] = -1
        data['Number_of_Trolleys'] = -1
    df = pd.DataFrame(data, index=[0])
    
    return df

In [3]:
def scrape_data(base_url, search_addon, page, ad_type,driver):
    
    additional_facts = pd.DataFrame(columns =['Posted_By', 'Object_Type', 'Condition', 'Heating', 'Floor', 'Max_Floors', 'Flag_Available_Immediately', 'Flag_Legal','Flag_Not_Last_Floor', 'Flag_Not_Part_of_House', 'Flag_VAT_return','Flag_24', 'Flag_Duplex', 'Flag_26', 'Flag_27', 'Flag_28','Flag_Energy_Saving', 'Flag_30', 'Flag_Balcony', 'Flag_32','Flag_Covered_Balcony', 'Flag_AirConditioning', 'Flag_Elevator','Flag_Has_Basement', 'Flag_37', 'Flag_Telephone', 'Flag_CableTV','Flag_Internet', 'Flag_Interphone', 'Flag_Surveliance', 'Flag_43','Flag_Garage', 'Flag_Parking', 'Number_of_Busses', 'Number_of_Trams','Number_of_Trolleys'])

    url = base_url+search_addon+str(page)
    raw_page = urlopen(url).read()
    urlopen(url).close()
    bs_page = BeautifulSoup(raw_page, 'html.parser')
    
    divs = bs_page.findAll('div',{'class','product-item product-list-item {} real-estates my-ad-placeholder'.format(ad_type)})
    # initialize empty lists
    publish_date, prices, currency, city, city_area, city_subarea, city_street, property_type, property_size, property_norooms, desc, link_to_property   = ([] for _ in range(12))

    if divs: # if list is not empty - to speed things up for empty segments
        for div in divs:

            publish_date.append(div.findAll('div',{'class','pi-img-wrapper-under'})[0].span.text)       # extract date of ad posting

            price = div.findAll('div',{'data-value','central-feature'})[0].span.text                    # price and currency info
            price = price.split('\xa0')

            prices.append(int(price[0].translate(str.maketrans({'.': '', ',': ''}))))
            currency.append(price[1])

            city_info = div.findAll('div',{'class','col-md-6 col-sm-5 col-xs-6 col-lg-6 sm-margin'})[0] # extract city, area and street

            city.append(city_info.find('ul',attrs={'class':'subtitle-places'}).text.split('\xa0')[0])
            city_area.append(city_info.find('ul',attrs={'class':'subtitle-places'}).text.split('\xa0')[1])
            city_subarea.append(city_info.find('ul',attrs={'class':'subtitle-places'}).text.split('\xa0')[2])
            city_street.append(city_info.find('ul',attrs={'class':'subtitle-places'}).text.split('\xa0')[3])

            features = div.find('ul',attrs={'class':'ad-features'})                                     # extract price, size and type of posting

            property_type.append(features.findAll('li',{'class':'col-p-1-3'})[0].div.text.split('\xa0')[0])
            property_size.append(features.findAll('li',{'class':'col-p-1-3'})[1].div.text.split('\xa0')[0])
            property_norooms.append(features.findAll('li',{'class':'col-p-1-3'})[2].div.text.split('\xa0')[0])

            try: 
                desc.append(div.findAll('p',{'class':'text-description-list ad-description short-desc'})[0].text)
            except:
                desc.append('')
            link_to_property.append(div.find('h3').a['href'])
    #        print(link_to_property[len(link_to_property)-1])
            tmp_additional_facts = subscrape_data(str(base_url+link_to_property[len(link_to_property)-1]),driver)

            additional_facts = additional_facts.append(tmp_additional_facts, ignore_index=True)

        df = pd.DataFrame(list(zip(publish_date,prices,currency,city,city_area,city_subarea,city_street,property_type,property_size,property_norooms,desc, link_to_property)), 
               columns =['Publish_Date', 'Price','Currency','City','City_Area','City_Subarea','Street','Property_Type','Property_Size_sqmtr','No_of_Rooms','Desc','Link_to_Property'])
        df['Ad_Type'] = ad_type
    else:
        df = pd.DataFrame(list(zip(publish_date,prices,currency,city,city_area,city_subarea,city_street,property_type,property_size,property_norooms,desc, link_to_property)), 
               columns =['Publish_Date', 'Price','Currency','City','City_Area','City_Subarea','Street','Property_Type','Property_Size_sqmtr','No_of_Rooms','Desc','Link_to_Property'])
        df['Ad_Type'] = None

    return pd.concat([df, additional_facts], axis=1,sort=True)

In [5]:
num_pages = 1478

for page in range(654,num_pages):
    driver = Chrome('chromedriver1.exe')
    print('page: ' + str(page))
#     tmp_df = scrape_data('https://www.halooglasi.com', '/nekretnine/prodaja-stanova?page=', page, 'Premium',driver)
#     print(len(tmp_df))
#     df = pd.concat([df, tmp_df], axis=0, ignore_index=True,sort=True)
#     tmp_df = scrape_data('https://www.halooglasi.com', '/nekretnine/prodaja-stanova?page=', page, 'Top',driver)
#     print(len(tmp_df))
#     df = pd.concat([df, tmp_df], axis=0, ignore_index=True,sort=True)
    tmp_df = scrape_data('https://www.halooglasi.com', '/nekretnine/prodaja-stanova?page=', page, 'Standard',driver)
    print(len(tmp_df))
    df = pd.concat([df, tmp_df], axis=0, ignore_index=True,sort=True)
    driver.close()

page: 654
20
page: 655
20
page: 656
20
page: 657
19
page: 658
18
page: 659
20
page: 660
20
page: 661
19
page: 662
18
page: 663
20
page: 664
20
page: 665
18
page: 666
20
page: 667
18
page: 668
20
page: 669
20
page: 670
20
page: 671
19
page: 672
20
page: 673
20
page: 674
20
page: 675
20
page: 676
20
page: 677
14
page: 678
19
page: 679
19
page: 680
20
page: 681
20
page: 682
20
page: 683
20
page: 684
20
page: 685
20
page: 686
20
page: 687
20
page: 688
20
page: 689
20
page: 690
20
page: 691
20
page: 692
20
page: 693
20
page: 694
20
page: 695
19
page: 696
18
page: 697
20
page: 698
20
page: 699
20
page: 700
20
page: 701
20
page: 702
20
page: 703
20
page: 704
18
page: 705
20
page: 706
19
page: 707
20
page: 708
20
page: 709
20
page: 710
20
page: 711
19
page: 712
19
page: 713
17
page: 714
19
page: 715
18
page: 716
20
page: 717
20
page: 718
19
page: 719
19
page: 720
20
page: 721
20
page: 722
20
page: 723
20
page: 724
20
page: 725
20
page: 726
20
page: 727
18
page: 728
20
page: 729
19
page: 730
20

TimeoutException: Message: timeout
  (Session info: chrome=78.0.3904.97)


In [6]:
df.drop_duplicates().to_csv('halooglasi_data_20191106.csv',index=False)

In [7]:
df = pd.read_csv('halooglasi_data_20191106.csv')

In [114]:
df.head()

,Ad_Type,City,City_Area,City_Subarea,Condition,Currency,Desc,Flag_24,Flag_26,Flag_27,...,Number_of_Busses,Number_of_Trams,Number_of_Trolleys,Object_Type,Posted_By,Price,Property_Size_sqmtr,Property_Type,Publish_Date,Street
0,Premium,Beograd,Opština Vračar,Krunska,Lux,€,AGENCIJSKA PROVIZIJA 2% OPIS NEKRETNINE: Uknji...,-1,-1,-1,...,3,0,4,NaN,Agencija,300000,108,Stan,06.11.2019,Kneginje Zorke
1,Premium,Beograd,Opština Zvezdara,Kluz,Izvorno stanje,€,"Dvoiposoban stan neto površine 65,00 m2 po cen...",-1,1,-1,...,0,0,0,Novogradnja,Investitor,103409,65,Stan,06.11.2019,Živka Davidovića 81
2,Premium,Beograd,Opština Vračar,Hram svetog Save,NaN,€,"HITNA PRODAJA! Hram, Patrijarha Varnave, 4.0, ...",-1,-1,-1,...,0,0,0,Novogradnja,Agencija,400000,185,Stan,06.11.2019,Patrijarha Varnave
3,Premium,Beograd,Opština Savski venac,Beograd na vodi,Lux,€,"Ekskluzivan, DELUXE stan, BEOGRAD NA VODI - zg...",-1,-1,-1,...,0,0,0,Novogradnja,Agencija,270000,80,Stan,NaN,Vista
4,Premium,Beograd,Opština Zemun,Donji grad,Izvorno stanje,€,Stan se nalazi u lameli 8 u IV fazi izgradnje ...,-1,-1,-1,...,0,0,0,Novogradnja,Investitor,267600,"127,44",Stan,06.11.2019,Petra Kočića


In [53]:
url = 'https://www.halooglasi.com/nekretnine/prodaja-stanova/zelena-avenija-zemun-novogradnja-pametne-zgra/5425634578403?kid=4&sid=1573042134158'

In [54]:
posted_by, object_type, condition, heating, floor, max_floors, flag_legal, flag_available_immediately, flag_not_last_floor,flag_balcony, flag_elevator, flag_internet, flag_surveliance, flag_air_condition, flag_cable_tv, flag_garage, flag_interphone, number_of_busses, number_of_trams, number_of_trolleys = ([] for _ in range(20))
driver = Chrome('chromedriver1.exe')

driver.get(url)
time.sleep(1)      # wait page to load, to grab raw values
page=driver.page_source
soup = BeautifulSoup(page, 'html.parser')

info = soup.findAll('div',attrs={'class':'col-lg-7 col-md-7 datasheet-features-type'})
driver.close()

In [92]:
driver = Chrome('chromedriver1.exe')
url = 'https://www.halooglasi.com/nekretnine/prodaja-stanova/zelena-avenija-zemun-novogradnja-pametne-zgra/5425634578403?kid=4&sid=1573042134158'
a = subscrape_data(url, driver)

In [93]:
driver.close()

In [44]:
df.head(2)

,Ad_Type,City,City_Area,City_Subarea,Condition,Currency,Desc,Flag_Air_Condition,Flag_Available_Immediately,Flag_Balcony,...,Num_of_Busses,Num_of_Trams,Num_of_Trolleys,Object_Type,Posted_By,Price,Property_Size_sqmtr,Property_Type,Publish_Date,Street
1,Premium,Beograd,Opština Zvezdara,Mali Mokri Lug,Izvorno stanje,€,Stambeni objekat se nalazi u Partizanskoj ulic...,1,1,1,...,9,0,0,Novogradnja,Investitor,45000.0,"36,39",Stan,02.11.2019,Partizanska
2,Premium,Beograd,Opština Zvezdara,Mali Mokri Lug,Izvorno stanje,€,Stambeni objekat se nalazi u Partizanskoj ulic...,1,1,1,...,9,0,0,Novogradnja,Investitor,50150.0,"40,15",Stan,02.11.2019,Partizanska


In [90]:
def subscrape_data(url, driver):
    # 12 - posted_by 
    # 13 - object_type 
    # 14 - condition 
    # 15 - heating 
    # 16 - floor 
    # 17 - max_floors

    # 19 - flag_available_immediately 
    # 20 - flag_legal 
    # 21 - flag_not_last_floor 
    # 22 - flag_not_in_house
    # 23 - flag_VAT_return
    # 25 - flag_duplex
    # 29 - flag_energy_saving
    # 31 - flag_balcony
    # 33 - flag_covered_balcony
    # 34 - flag_air_condition
    # 35 - flag_elevator
    # 36 - flag_has_basement
    # 38 - flag_telephone
    # 39 - flag_cable_tv
    # 40 - flag_internet
    # 41 - flag_interphone
    # 42 - flag_surveliance
    # 44 - flag_garage
    # 45 - flag_parking
    

    driver.get(url)
    time.sleep(1)      # wait page to load, to grab raw values
    page=driver.page_source
    soup = BeautifulSoup(page, 'html.parser')

    data = {}

    
    try:    
        data['Posted_By'] = soup.find('span',attrs={'id':'plh12'}).text
    except:
        data['Posted_By'] = ''
    try:    
        data['Object_Type'] = soup.find('span',attrs={'id':'plh13'}).text
    except:
        data['Object_Type'] = ''
    try:    
        data['Condition'] = soup.find('span',attrs={'id':'plh14'}).text
    except:
        data['Condition'] = ''
    try:    
        data['Heating'] = soup.find('span',attrs={'id':'plh15'}).text
    except:
        data['Heating'] = ''
    try:    
        data['Floor'] = str(soup.find('span',attrs={'id':'plh16'}).text)
    except:
        data['Floor'] = '-1'
    try:    
        data['Max_Floors'] = str(soup.find('span',attrs={'id':'plh17'}).text)
    except:
        data['Max_Floors'] = '-1'
    
        
    # flags part
    try:
        soup.find('span',attrs={'id':'plh19'}).text
        data['Flag_Available_Immediately'] = 1
    except:
        data['Flag_Available_Immediately'] = -1
    
    try:
        soup.find('span',attrs={'id':'plh20'}).text
        data['Flag_Legal'] = 1
    except:
        data['Flag_Legal'] = -1
    try:
        soup.find('span',attrs={'id':'plh21'}).text
        data['Flag_Not_Last_Floor'] = 1
    except:
        data['Flag_Not_Last_Floor'] = -1
    try:
        soup.find('span',attrs={'id':'plh22'}).text
        data['Flag_Not_Part_of_House'] = 1
    except:
        data['Flag_Not_Part_of_House'] = -1
    try:
        soup.find('span',attrs={'id':'plh23'}).text
        data['Flag_VAT_return'] = 1
    except:
        data['Flag_VAT_return'] = -1
    try:
        soup.find('span',attrs={'id':'plh24'}).text
        data['Flag_24'] = 1
    except:
        data['Flag_24'] = -1
    try:
        soup.find('span',attrs={'id':'plh25'}).text
        data['Flag_Duplex'] = 1
    except:
        data['Flag_Duplex'] = -1
    try:
        soup.find('span',attrs={'id':'plh26'}).text
        data['Flag_26'] = 1
    except:
        data['Flag_26'] = -1
    try:
        soup.find('span',attrs={'id':'plh27'}).text
        data['Flag_27'] = 1
    except:
        data['Flag_27'] = -1
    try:
        soup.find('span',attrs={'id':'plh28'}).text
        data['Flag_28'] = 1
    except:
        data['Flag_28'] = -1
    try:
        soup.find('span',attrs={'id':'plh29'}).text
        data['Flag_Energy_Saving'] = 1
    except:
        data['Flag_Energy_Saving'] = -1
    try:
        soup.find('span',attrs={'id':'plh30'}).text
        data['Flag_30'] = 1
    except:
        data['Flag_30'] = -1
    try:
        soup.find('span',attrs={'id':'plh31'}).text
        data['Flag_Balcony'] = 1
    except:
        data['Flag_Balcony'] = -1
    try:
        soup.find('span',attrs={'id':'plh32'}).text
        data['Flag_32'] = 1
    except:
        data['Flag_32'] = -1
    try:
        soup.find('span',attrs={'id':'plh33'}).text
        data['Flag_Covered_Balcony'] = 1
    except:
        data['Flag_Covered_Balcony'] = -1
    try:
        soup.find('span',attrs={'id':'plh34'}).text
        data['Flag_AirConditioning'] = 1
    except:
        data['Flag_AirConditioning'] = -1
    try:
        soup.find('span',attrs={'id':'plh35'}).text
        data['Flag_Elevator'] = 1
    except:
        data['Flag_Elevator'] = -1
    try:
        soup.find('span',attrs={'id':'plh36'}).text
        data['Flag_Has_Basement'] = 1
    except:
        data['Flag_Has_Basement'] = -1
    try:
        soup.find('span',attrs={'id':'plh37'}).text
        data['Flag_37'] = 1
    except:
        data['Flag_37'] = -1
    try:
        soup.find('span',attrs={'id':'plh38'}).text
        data['Flag_Telephone'] = 1
    except:
        data['Flag_Telephone'] = -1
    try:
        soup.find('span',attrs={'id':'plh39'}).text
        data['Flag_CableTV'] = 1
    except:
        data['Flag_CableTV'] = -1
    try:
        soup.find('span',attrs={'id':'plh40'}).text
        data['Flag_Internet'] = 1
    except:
        data['Flag_Internet'] = -1
    try:
        soup.find('span',attrs={'id':'plh41'}).text
        data['Flag_Interphone'] = 1
    except:
        data['Flag_Interphone'] = -1
    try:
        soup.find('span',attrs={'id':'plh42'}).text
        data['Flag_Surveliance'] = 1
    except:
        data['Flag_Surveliance'] = -1
    try:
        soup.find('span',attrs={'id':'plh43'}).text
        data['Flag_43'] = 1
    except:
        data['Flag_43'] = -1
    try:
        soup.find('span',attrs={'id':'plh44'}).text
        data['Flag_Garage'] = 1
    except:
        data['Flag_Garage'] = -1
    try:
        soup.find('span',attrs={'id':'plh45'}).text
        data['Flag_Parking'] = 1
    except:
        data['Flag_Parking'] = -1
    try:    
        data['Number_of_Busses'] = len(soup.findAll('div',attrs={'class':'city-lines'})[0].findAll('li',attrs={'class':'busses'}))
        data['Number_of_Trams'] = len(soup.findAll('div',attrs={'class':'city-lines'})[0].findAll('li',attrs={'class':'trams'}))
        data['Number_of_Trolleys'] = len(soup.findAll('div',attrs={'class':'city-lines'})[0].findAll('li',attrs={'class':'trolleybus'})) 
    except:
        data['Number_of_Busses'] = -1
        data['Number_of_Trams'] = -1
        data['Number_of_Trolleys'] = -1
    df = pd.DataFrame(data, index=[0])
    
    return df